## Butanol Production Pathway: Attractor Bifurcation Analysis

In this notebook, we analyse the attractors of a butanol production pathway model as appearing on [CellCollective]( https://research.cellcollective.org/?dashboard=true#module/36604:1/signaling-pathway-for-butanol-production-in-clostridium-beijerinckii-nrrl-b598/10) \[1]. The model has been exported as `.sbml` and is included in this project as `butanol-pathway.sbml`. However, this `butanol-pathway.sbml` file cannot be processed by AEON directly, since it contains logical inconsistencies between update functions and the regulatory graph. We thus also include a version of the model where the problematic requirements about the regulatory graph of the network have been removed (`butanol-pathway-relaxed.aeon`).

In this notebook, when we refer to the measurements or observations about the system performed by the original authors, we primarily mean \[1]. However, \[1] also relies on the data and measurements from \[2] and \[3].

### Attractor bifurcation analysis

In the following code, we demonstrate the problem with the original model and show how it has been fixed in the relaxed version of the regulatory graph:

In [1]:
from biodivine_aeon import *
from pandas import DataFrame # Only used for visualising tables.
from pathlib import Path

Detected IPython (`ZMQInteractiveShell`). Log level set to `LOG_ESSENTIAL`.


In [2]:
original_model = BooleanNetwork.from_file('butanol-pathway.sbml')
model = BooleanNetwork.from_file('butanol-pathway-relaxed.aeon')
print(model)

BooleanNetwork(variables=66, regulations=139, explicit_parameters=0, implicit_parameters=13)


In [3]:
# Try to build a symbolic asynchronous graph from the original model.
# This should list problems in the structure of the original model.
try:
    graph = AsynchronousGraph(original_model)
except Exception as e:
    print("Cannot create graph from the original model.")
    print(e)

Cannot create graph from the original model.
No update functions satisfy given constraints: 
 - spoIIE not activating in spoIIAB.
 - sigA has no effect in spo0A_p.
 - sporulation has no effect in glucose___PTS.



In [4]:
# Now print regulations that we changed to make the model logically consistent.

print("==== spoIIE -> spoIIAB updated from activation to inhibition ====")
print(original_model.find_regulation("spoIIE", "spoIIAB"))
print(model.find_regulation("spoIIE", "spoIIAB"))

print("==== sigA -> spo0A_p observability is removed ====")
print(original_model.find_regulation("sigA", "spo0A_p"))
print(model.find_regulation("sigA", "spo0A_p"))

print("==== sporulation -> glucose___PTS observability is removed ====")
print(original_model.find_regulation("sporulation", "glucose___PTS"))
print(model.find_regulation("sporulation", "glucose___PTS"))

==== spoIIE -> spoIIAB updated from activation to inhibition ====
{'source': VariableId(63), 'target': VariableId(60), 'essential': True, 'sign': '+'}
{'source': VariableId(63), 'target': VariableId(60), 'essential': True, 'sign': '-'}
==== sigA -> spo0A_p observability is removed ====
{'source': VariableId(51), 'target': VariableId(58), 'essential': True, 'sign': '+'}
{'source': VariableId(51), 'target': VariableId(58), 'essential': False, 'sign': '+'}
==== sporulation -> glucose___PTS observability is removed ====
{'source': VariableId(64), 'target': VariableId(39), 'essential': True, 'sign': '-'}
{'source': VariableId(64), 'target': VariableId(39), 'essential': False, 'sign': '-'}


In [5]:
# Note that we can achieve similar effect by letting AEON infer the regulations automatically.
# However, this process completely removes the two non-essential regulations, since they
# play not role in the network. In our manually edited version, we chose to keep these
# regulations but designated them as non-essential, which means they do not need to play
# a role in the target update function.
inferred = original_model.infer_valid_graph()
print(inferred.find_regulation("spoIIE", "spoIIAB"))
print(inferred.find_regulation("sigA", "spo0A_p"))
print(inferred.find_regulation("sporulation", "glucose___PTS"))

{'source': VariableId(63), 'target': VariableId(60), 'essential': True, 'sign': '-'}
None
None


Finally, the model that we just loaded contains inputs. We can automatically turn these inputs into logical parameters. This simplifies many computations, as it allows AEON to encode the model more succinctly.

In [6]:
model = model.inline_inputs(infer_inputs=True, repair_graph=True)
print(model)

# And we can succssfully create a symbolic graph from the relaxed model:
graph = AsynchronousGraph(model)
print(graph.mk_unit_vertices(), graph.mk_unit_colors())
print(model.explicit_parameter_names())

BooleanNetwork(variables=53, regulations=117, explicit_parameters=13, implicit_parameters=0)
VertexSet(cardinality=9007199254740992, symbolic_size=2) ColorSet(cardinality=8192, symbolic_size=2)
['NADH', 'NAD_P_H', 'PTS', 'Rnf', 'fba', 'gap_pgk_tpi_pgm__X276_23705_eno', 'glucose', 'pfk', 'pfo', 'pgi', 'phosphorylation', 'sigA', 'spoIIE']


Now, we can compute the attractors of this model for every possible combination of its parameter values (i.e. colors), which in this case are the valuations of the network inputs. Importantly, we can verify that for every parametrisation, there is exactly one attractor. Therefore, we can safely merge the two discovered sets of states into one attractor set.

> Due to the way the on-the-fly attractor detection algorithm works, it can output multiple sets of attractors even when they in fact cover disjoint parameterisations.

In [7]:
# This can take up several seconds to compute.
attractors = Attractors.attractors(graph)

# Check that we indeed got two disjoint attractor sets.
assert(len(attractors) == 2)
assert(attractors[0].colors().intersect(attractors[1].colors()).is_empty())

# Merge the two disjoint sets into a single attractor set.
attractor = attractors[0].union(attractors[1])

Start interleaved transition guided reduction with 73786976294838210000[nodes:2] candidates.
 > Discarded 36893488147419103000 instances based on the BnVariable(43) extended component.
 > State space reduced to 36893488147419103000[nodes:5].
 > Discarded 18446744073709552000 instances based on the BnVariable(38) extended component.
 > State space reduced to 18446744073709552000[nodes:8].
 > Discarded 9223372036854776000 instances based on the BnVariable(31) extended component.
 > State space reduced to 9223372036854776000[nodes:11].
 > Discarded 4611686018427388000 instances based on the BnVariable(2) extended component.
 > State space reduced to 4611686018427388000[nodes:14].
 > Discarded 1130403506469994500 instances based on the BnVariable(21) extended component.
 > State space reduced to 3481282511957393400[nodes:55].
 > Discarded 1583015269020729300 instances based on the BnVariable(37) extended component.
 > State space reduced to 1898267242936664000[nodes:97].
 > Discarded 45035

Analysing this model in the AEON user interface allows us to construct a basic decision tree describing how the behaviour of the model (i.e. the discovered attractors) changes for different settings of logical parameters. Since this is to a large extent an interactive task, it is not yet supported in the Python API. For completness, we here give a screenshot of the decision tree, which will inform our subsequent steps. Note that we refer to the three leafs of the tree as mode **I**, **II**, and **III** (left to right). 

<p align="center">
  <img src="./decision-tree.png" width="300" />
</p>

### Validating known observations

To compare our attractors with existing knowledge about the pathway, we need to further explore how the individual variables of the model behave in each of the three modes (leafs of the decision tree) shown above. We can do this using simple queries about the structure of the `attractor` set:

In [8]:
# Function `mk_colors` fixes the interpretation of a single unknown function/parameter.
# Since the parameter in question is an input (arity zero), the only possible interpretations
# are "true" and "false".
sigA_off = graph.mk_function_colors("sigA", "false")
sigA_on = graph.mk_function_colors("sigA", "true")

glucose_off = graph.mk_function_colors("glucose", "false")
glucose_on = graph.mk_function_colors("glucose", "true")

mode_I = attractor.intersect_colors(sigA_off)
mode_II = attractor.intersect_colors(sigA_on).intersect_colors(glucose_off)
mode_III = attractor.intersect_colors(sigA_on).intersect_colors(glucose_on)

print(int(mode_I.cardinality()), int(mode_II.cardinality()), int(mode_III.cardinality()))

112508022784 2048 867621814984704


In [9]:
# Classify the colors depending on whether a variable can be always true, 
# always false, or change its value (i.e. the set states contains both instances where the 
# variable is true and false).
def check_stability(graph, states, variable):
    on_colors = states.intersect(graph.mk_subspace({ variable: True})).colors()
    off_colors = states.intersect(graph.mk_subspace({ variable: False })).colors()
    both = on_colors.intersect(off_colors)
    return { "on": on_colors.minus(both), "off": off_colors.minus(both), "both": both }

print("Mode I", check_stability(graph, mode_I, "butanol"))
print("Mode II", check_stability(graph, mode_II, "butanol"))
print("Mode III", check_stability(graph, mode_III, "butanol"))

Mode I {'on': ColorSet(cardinality=3072, symbolic_size=6), 'off': ColorSet(cardinality=0, symbolic_size=1), 'both': ColorSet(cardinality=1024, symbolic_size=5)}
Mode II {'on': ColorSet(cardinality=0, symbolic_size=1), 'off': ColorSet(cardinality=2048, symbolic_size=4), 'both': ColorSet(cardinality=0, symbolic_size=1)}
Mode III {'on': ColorSet(cardinality=0, symbolic_size=1), 'off': ColorSet(cardinality=0, symbolic_size=1), 'both': ColorSet(cardinality=2048, symbolic_size=4)}


Here, we see that in mode **I**, `butanol` production is either perpetually (*on*) or intermittently (*both*) enabled. Meanwhile, in mode **II**, the value of `butanol` remains `false` for every possible parametrisation. Finally, in mode **III**, the value changes between `true` and `false` but never stabilises to a fixed value. Since we know from the decision tree that this attractor is disordered, we can assume that this is not a periodic oscillation.

For comparison, we can run this analysis for multiple variables to get a more complete picture about the state of the cell in each of the attractors. To make the data more readable, we transform the absolute numbers of parametrisations to relative percentages:

In [10]:
table = []

def process_stability(data):
    size_all = data["on"].cardinality() + data["off"].cardinality() + data["both"].cardinality()
    result = {}
    if not data["on"].is_empty():
        result["on"] = round((data["on"].cardinality() / size_all) * 100.0)
    if not data["off"].is_empty():
        result["off"] = round((data["off"].cardinality() / size_all) * 100.0)
    if not data["both"].is_empty():
        result["both"] = round((data["both"].cardinality() / size_all) * 100.0)
    return result

def add_row(graph, modes, table, variable):
    results = [ process_stability(check_stability(graph, mode, variable)) for mode in modes ]
    table.append([variable] + results)
    

for variable in ["acetic_acid", "butyric_acid", "lactic_acid", "butanol", "ethanol", "acetone", "sporulation"]:
    add_row(graph, [mode_I, mode_II, mode_III], table, variable)
    
DataFrame(table, columns=["variable", "sigA=off (I)", "glucose=off (II)", "glucose=on (III)"])

,variable,sigA=off (I),glucose=off (II),glucose=on (III)
0,acetic_acid,"{'on': 75, 'both': 25}",{'off': 100},{'both': 100}
1,butyric_acid,{'both': 100},{'off': 100},{'both': 100}
2,lactic_acid,"{'on': 35, 'off': 53, 'both': 12}",{'off': 100},"{'off': 52, 'both': 48}"
3,butanol,"{'on': 75, 'both': 25}",{'off': 100},{'both': 100}
4,ethanol,"{'on': 50, 'off': 50}",{'off': 100},"{'off': 75, 'both': 25}"
5,acetone,"{'on': 75, 'both': 25}",{'off': 100},{'both': 100}
6,sporulation,"{'off': 75, 'both': 25}",{'on': 100},{'both': 100}


We focus on modes **II** and **III** which are studied in \[1]. Here, the attractors align with biological expectations: when the nutrient (`glucose`) is absent, `sporulation` is active, and subsequently, production of other entities is supressed. Meanwhile, with nutrients, the cell is guaranteed to produce (irregularly) all products except for `ethanol` and `lactic_acid`. For these two products, there are possible input valuations where the production does not occur. For `ethanol`, this is expected. For `lactic_acid`, we also see relatively low expression in the observed data, but it may warrant additional investigation.

Below, you can compare our data with in-vivo measurements and simulation performed as part of \[1].

<p align="center">
  <img src="./butanol-measurements.png" width="400" />
</p>

<p align="center">
  <img src="./butanol-simulation.png" width="300" />
</p>

Furthermore, we can also compute the activity of other variables, in particular genes that were ivestigated in \[1]:

In [11]:
gene_table = []

for variable in ["ack", "adc", "bdhAB", "buk1", "crt", "hbd", "pta"]:
    add_row(graph, [mode_I, mode_II, mode_III], gene_table, variable)

DataFrame(gene_table, columns=["variable", "sigA=off (I)", "glucose=off (II)", "glucose=on (III)"])

,variable,sigA=off (I),glucose=off (II),glucose=on (III)
0,ack,"{'on': 75, 'both': 25}",{'off': 100},{'both': 100}
1,adc,"{'off': 75, 'both': 25}",{'off': 100},"{'off': 50, 'both': 50}"
2,bdhAB,"{'off': 75, 'both': 25}",{'off': 100},"{'off': 50, 'both': 50}"
3,buk1,"{'on': 75, 'both': 25}",{'off': 100},{'both': 100}
4,crt,"{'on': 75, 'both': 25}",{'off': 100},{'both': 100}
5,hbd,"{'on': 75, 'both': 25}",{'off': 100},{'both': 100}
6,pta,"{'on': 75, 'both': 25}",{'off': 100},{'both': 100}


Here, we also find that we are mostly in aggreement with the behaviour measured by the authors, where activity of all genes is suppressed in the sporulation stage, but each gene should become (irregularly) active once `glucose` is present. The only exception are `adc` and `bdhAB`, which also allow configurations with no activity. Note that these two genes only appear to activate after a period of time in the original experiments, indicating a change in the environment which could align with a change of the input parameters in our model.

### Update function repair

In the first part, we had to relax the regulatory graph of the network because it was not consistent with its update functions. Now, we attempt to fix this issue by devising a new version of the model that is consistent with the original regulatory graph, but also reproduces the attractors of the original network as closely as possible (since these were experimentally confirmed).

As the first step, we use the original Boolean model, but augment it with three uninterpreted Boolean functions which will stand in for the (currently) problematic parts of the update functions. Such a change would be easier to perform in the interactive model editor provided by AEON, but for completeness, we also describe it here in code:

In [12]:
model = BooleanNetwork.from_file('butanol-pathway.sbml')

# Declare uninterpreted functions:
model.add_explicit_parameter("f_1", 2 )
model.add_explicit_parameter("f_2", 2 )
model.add_explicit_parameter("f_3", 3 )

# Change original functions and confirm that the change has been successful:
f_spo0A_p = "spo0A & phosphorylation & f_1(sigA, sporulation)"
f_glucose_PTS = "glucose & PTS & f_2(cell_membrane, sporulation)"
f_spoIIAB = "spo0A_p & f_3(sigH, spoIIAA, spoIIE)"

model.set_update_function("spo0A_p", f_spo0A_p)
model.set_update_function("glucose___PTS", f_glucose_PTS)
model.set_update_function("spoIIAB", f_spoIIAB)

print(model.get_update_function("spo0A_p"))

spo0A & phosphorylation & f_1(sigA, sporulation)


Note that in this case, we have not inlined the network inputs as parameters. This is because some of the inputs are arguments of the newly added uninterpreted functions, and we want to maintain the annotations regarding essentiality and monotonicity of those influences. If we turned them into parameters, we lose the ability to assign those requirements.

 > Such annotations for parameters are currently not supported, but we are intending to add them in the future.

To better understand the possible behaviours of our parametrized model, we can print the individual functions that are valid for each of the uninterpreted symbols:

In [13]:
graph = AsynchronousGraph(model)
for m in graph.mk_unit_colors().items(["f_1"]):
    print("f_1:", m["f_1"])
    print("spo0A_p update:", m.instantiate(model.get_update_function("spo0A_p")))

print("")
for m in graph.mk_unit_colors().items(["f_2"]):
    print("f_2:", m["f_2"])
    print("glucose___PTS update:", m.instantiate(model.get_update_function("glucose___PTS")))

print()
for m in graph.mk_unit_colors().items(["f_3"]):
    print("f_3:", m["f_3"])
    print("spoIIAB update:", m.instantiate(model.get_update_function("spoIIAB")))

f_1: (x_0 & !x_1)
spo0A_p update: phosphorylation & sigA & spo0A & !sporulation
f_1: ((!x_0 & !x_1) | x_0)
spo0A_p update: (phosphorylation & sigA & spo0A) | (phosphorylation & !sigA & spo0A & !sporulation)

f_2: (x_0 & !x_1)
glucose___PTS update: PTS & cell_membrane & glucose & !sporulation
f_2: (!x_0 & !x_1)
glucose___PTS update: PTS & !cell_membrane & glucose & !sporulation
f_2: ((!x_0 & !x_1) | x_0)
glucose___PTS update: (PTS & cell_membrane & glucose) | (PTS & !cell_membrane & glucose & !sporulation)
f_2: (!x_0 | (x_0 & !x_1))
glucose___PTS update: (PTS & cell_membrane & glucose & !sporulation) | (PTS & !cell_membrane & glucose)

f_3: (x_0 & (!x_1 & x_2))
spoIIAB update: sigH & spo0A_p & !spoIIAA & spoIIE
f_3: ((!x_0 & (!x_1 & x_2)) | (x_0 & x_2))
spoIIAB update: (sigH & spo0A_p & spoIIE) | (!sigH & spo0A_p & !spoIIAA & spoIIE)
f_3: ((x_0 & !x_1) | (x_0 & (x_1 & x_2)))
spoIIAB update: (sigH & spo0A_p & spoIIAA & spoIIE) | (sigH & spo0A_p & !spoIIAA)
f_3: ((!x_0 & (!x_1 & x_2)) | (

In [14]:
# Save the modified model for future reference.
Path('butanol-pathway-f1-f2-f3.aeon').write_text(model.to_aeon());

With these new explicit "functional" parameters, we can again compute the attractors of the network, and just as before, we end up with a single attractor. *Note that the compuation can take several minutes.* This is because AEON actually tests every combination of possible functions `f_1`, `f_2`, and `f_3`, as long as they satisfy the constraints given by the regulatory graph.

In [15]:
graph = AsynchronousGraph(model)
attractors = Attractors.attractors(graph, graph.mk_unit_colored_vertices())

assert(len(attractors) == 2)
assert(attractors[0].colors().intersect(attractors[1].colors()).is_empty())

attractor = attractors[0].union(attractors[1])

Start interleaved transition guided reduction with 43521329506126650000000000[nodes:39] candidates.
 > Discarded 21760664753063325000000000 instances based on the BnVariable(63) extended component.
 > State space reduced to 21760664753063325000000000[nodes:42].
 > Discarded 10880332376531663000000000 instances based on the BnVariable(51) extended component.
 > State space reduced to 10880332376531663000000000[nodes:45].
 > Discarded 5440166188265831000000000 instances based on the BnVariable(55) extended component.
 > State space reduced to 5440166188265831000000000[nodes:47].
 > Discarded 2720083094132915600000000 instances based on the BnVariable(46) extended component.
 > State space reduced to 2720083094132915600000000[nodes:50].
 > Discarded 1360041547066457800000000 instances based on the BnVariable(45) extended component.
 > State space reduced to 1360041547066457800000000[nodes:53].
 > Discarded 680020773533228900000000 instances based on the BnVariable(44) extended component.


Interestingly, this (parametrised) set of attractors leads to the same bifurcation decision tree as we saw in the original model (see above).

Subsequently, we can also compute a similar table of stable and unstable variables as before:

In [16]:
# Here, we did not inline the network inputs. Hence, `sigA` and `glucose` 
# are variables, not parameters and we have to use `mk_subspace` instead of `mk_colors`.

sigA_off = graph.mk_subspace({ "sigA": False })
sigA_on = graph.mk_subspace({ "sigA": True })

glucose_off = graph.mk_subspace( { "glucose": False })
glucose_on = graph.mk_subspace({ "glucose": True })

mode_I = attractor.intersect(sigA_off)
mode_II = attractor.intersect(sigA_on).intersect(glucose_off)
mode_III = attractor.intersect(sigA_on).intersect(glucose_on)

print(int(mode_I.cardinality()), int(mode_II.cardinality()), int(mode_III.cardinality()))

10350737997824 147456 24311321730221696


In [17]:
table = []

for variable in ["acetic_acid", "butyric_acid", "lactic_acid", "butanol", "ethanol", "acetone", "sporulation"]:
    add_row(graph, [mode_I, mode_II, mode_III], table, variable)
    
DataFrame(table, columns=["variable", "sigA=off (I)", "glucose=off (II)", "glucose=on (III)"])

,variable,sigA=off (I),glucose=off (II),glucose=on (III)
0,acetic_acid,"{'on': 88, 'both': 12}",{'off': 100},"{'off': 12, 'both': 88}"
1,butyric_acid,{'both': 100},{'off': 100},"{'off': 12, 'both': 88}"
2,lactic_acid,"{'on': 42, 'off': 53, 'both': 6}",{'off': 100},"{'off': 52, 'both': 48}"
3,butanol,"{'on': 88, 'both': 12}",{'off': 100},{'both': 100}
4,ethanol,"{'on': 50, 'off': 50}",{'off': 100},"{'off': 90, 'both': 10}"
5,acetone,"{'on': 88, 'both': 12}",{'off': 100},{'both': 100}
6,sporulation,"{'off': 88, 'both': 12}",{'on': 100},{'both': 100}


This table is not that much different from the one we saw for the original model. Mostly, we only see a shift in the percentages. This is not a strong indicator of a real change if we don't know the actual real-world distribution of parametrisations (we could easily pollute the parameter space with parameters which are biologically unrealistic but skew the percentage towards a certain outcome). However, we see that in mode **III**, `acetic_acid` and `butyric_acid` can now become permanently inactive. This is something that isn't supported by the original observations, and as such we would generally like to avoid this behaviour.

To do so, we can inspect the set of parametrisations to see whether there are some candidate functions which would lead us to the desired outcome. There are many ways of doing this. In this case, we enumeration the functions in the symbolic set. That is, we go through all possible elements of the set, but with a projection to a specific part of the Boolean network. We have already done this on the "unit" set to show the possible interpretations of the newly declared parameters.

In [18]:
# Compute stability analysis for the acetic acid.
acetic_acid_stability = check_stability(graph, mode_III, "acetic_acid")
print(acetic_acid_stability)

{'on': ColorSet(cardinality=0, symbolic_size=1), 'off': ColorSet(cardinality=18432, symbolic_size=40), 'both': ColorSet(cardinality=129024, symbolic_size=53)}


In [19]:
# Since there are no "on" colors, we only wish to retain "both" and avoid "off".
desired_colors = acetic_acid_stability["both"]
undesired_colors = acetic_acid_stability["off"]

# Next, we make a projection of these color sets to each of the three variables with
# unknown functions to see how many possible update functions there are for each of the
# uninterpreted functions we declared earlier.

for fn, var in [("f_1", "spo0A_p"), ("f_2", "glucose___PTS"), ("f_3", "spoIIAB")]:
    print(f"Desired interpretations of {fn}:")
    for m in desired_colors.items([fn]):
        print(" >", m[fn], " instantiates as ", m.instantiate(model.get_update_function(var)))
    print(f"Undesired interpretations of {fn}:")
    for m in undesired_colors.items([fn]):
        print(" >", m[fn], " instantiates as ", m.instantiate(model.get_update_function(var)))

Desired interpretations of f_1:
 > (x_0 & !x_1)  instantiates as  phosphorylation & sigA & spo0A & !sporulation
 > ((!x_0 & !x_1) | x_0)  instantiates as  (phosphorylation & sigA & spo0A) | (phosphorylation & !sigA & spo0A & !sporulation)
Undesired interpretations of f_1:
 > ((!x_0 & !x_1) | x_0)  instantiates as  (phosphorylation & sigA & spo0A) | (phosphorylation & !sigA & spo0A & !sporulation)
Desired interpretations of f_2:
 > (x_0 & !x_1)  instantiates as  PTS & cell_membrane & glucose & !sporulation
 > (!x_0 & !x_1)  instantiates as  PTS & !cell_membrane & glucose & !sporulation
 > ((!x_0 & !x_1) | x_0)  instantiates as  (PTS & cell_membrane & glucose) | (PTS & !cell_membrane & glucose & !sporulation)
 > (!x_0 | (x_0 & !x_1))  instantiates as  (PTS & cell_membrane & glucose & !sporulation) | (PTS & !cell_membrane & glucose)
Undesired interpretations of f_2:
 > (x_0 & !x_1)  instantiates as  PTS & cell_membrane & glucose & !sporulation
 > (!x_0 & !x_1)  instantiates as  PTS & !cel

From this list, we can see that for `f_2` and `f_3`, all functions that appear in the "desired" color set also appear in the "undesired" color set. This means that the dependence between these two functions and the outcome is either trivial (it does not depend on them at all), or more complex than what we can see in this list (only specific combinations of the functions lead to the desired outcome). Subsequently, we could try to enumerate a projection to both `f_2` and `f_3` at the same time to decide which of these is the actual reality.

However, we can also notice that for `f_!`, function `(x_0 & !x_1)` guarantees the desired outcome (i.e. it does not appear in the undesired list at all). As such, we can try to simply fix the update function accordingly to guarantee that `acetic_acid` cannot stabilise in the "off" state anymore.

> Note that using this transformation, we are losing some information. There is a second viable update function for `spo0A_p`, which in this case, we can't tell too much about. However, since our goal is to repair the original model without any new observations or data, using the simplest possible solution is a reasonable initial position, since any of the networks in the `desired_colors` would satisfy our requirement.

As such, we re-initialize the model with the new update function and continue our analysis:

In [20]:
# Reload model again
model = BooleanNetwork.from_sbml(Path('butanol-pathway.sbml').read_text())

# Do not add f_1 any more
model.add_explicit_parameter("f_2", 2)
model.add_explicit_parameter("f_3", 3)

# Re-declare update functions, using the new fixed f_1
f_spo0A_p = "spo0A & phosphorylation & sigA & !sporulation"
f_glucose_PTS = "glucose & PTS & f_2(cell_membrane, sporulation)"
f_spoIIAB = "spo0A_p & f_3(sigH, spoIIAA, spoIIE)"

model.set_update_function("spo0A_p", f_spo0A_p)
model.set_update_function("glucose___PTS", f_glucose_PTS)
model.set_update_function("spoIIAB", f_spoIIAB)

print(model.get_update_function("spo0A_p"))

spo0A & phosphorylation & sigA & !sporulation


In [21]:
# Save the modified model for future reference.
Path('butanol-pathway-f2-f3.aeon').write_text(model.to_aeon());

In [22]:
# Recompute the attractor set (this time the computation is slightly
# faster since we removed one of the unknown functions)
graph = AsynchronousGraph(model)
attractors = Attractors.attractors(graph)

assert(len(attractors) == 2)
assert(attractors[0].colors().intersect(attractors[1].colors()).is_empty())

attractor = attractors[0].union(attractors[1])

Start interleaved transition guided reduction with 21760664753063325000000000[nodes:32] candidates.
 > Discarded 10880332376531663000000000 instances based on the BnVariable(63) extended component.
 > State space reduced to 10880332376531663000000000[nodes:35].
 > Discarded 5440166188265831000000000 instances based on the BnVariable(51) extended component.
 > State space reduced to 5440166188265831000000000[nodes:38].
 > Discarded 2720083094132915600000000 instances based on the BnVariable(55) extended component.
 > State space reduced to 2720083094132915600000000[nodes:40].
 > Discarded 1360041547066457800000000 instances based on the BnVariable(46) extended component.
 > State space reduced to 1360041547066457800000000[nodes:43].
 > Discarded 680020773533228900000000 instances based on the BnVariable(45) extended component.
 > State space reduced to 680020773533228900000000[nodes:46].
 > Discarded 340010386766614460000000 instances based on the BnVariable(44) extended component.
 > S

In [23]:
# Recompute the sets corresponding to different modes:
sigA_off = graph.mk_subspace({ "sigA": False })
sigA_on = graph.mk_subspace({ "sigA": True })

glucose_off = graph.mk_subspace({ "glucose": False })
glucose_on = graph.mk_subspace({ "glucose": True })

mode_I = attractor.intersect(sigA_off)
mode_II = attractor.intersect(sigA_on).intersect(glucose_off)
mode_III = attractor.intersect(sigA_on).intersect(glucose_on)

In [24]:
# Update our "stability analysis" table.
table = []

for variable in ["acetic_acid", "butyric_acid", "lactic_acid", "butanol", "ethanol", "acetone", "sporulation"]:
    add_row(graph, [mode_I, mode_II, mode_III], table, variable)
    
DataFrame(table, columns=["variable", "sigA=off (I)", "glucose=off (II)", "glucose=on (III)"])

,variable,sigA=off (I),glucose=off (II),glucose=on (III)
0,acetic_acid,{'on': 100},{'off': 100},{'both': 100}
1,butyric_acid,{'both': 100},{'off': 100},{'both': 100}
2,lactic_acid,"{'on': 47, 'off': 53}",{'off': 100},"{'off': 52, 'both': 48}"
3,butanol,{'on': 100},{'off': 100},{'both': 100}
4,ethanol,"{'on': 50, 'off': 50}",{'off': 100},"{'off': 81, 'both': 19}"
5,acetone,{'on': 100},{'off': 100},{'both': 100}
6,sporulation,{'off': 100},{'on': 100},{'both': 100}


From our stability analysis table, we can see that just as we expected, the inactivity of `acetic_acid` (and `butyric_acid`) in mode **III** is completely eliminated. However, if we compare this table to the previous two tables we obtained, it is also immediately obvious that this change eliminated the possibility of sporulation in mode **I**, leading to a much smaller variability of possible behaviour in this mode. In fact, the possible attractors in this mode are almost completely stable: the only variable that isn't stable is `butyric_acid` (and `butyrate`, not pictured).

However, this makes perfect sense, because our new update function for `spo0A_p` now actually requires `sigA` to be active in order to initiate sporulation, which was not the case before. Hence when `sigA = off`, sporulation becomes impossible with this new update function. This is warranted, since `sigA` is a known driver of sporulation \[4].

We can also similarly explore the expression of different significant genes, just as we did before:

In [25]:
gene_table = []

for variable in ["ack", "adc", "bdhAB", "buk1", "crt", "hbd", "pta"]:
    add_row(graph, [mode_I, mode_II, mode_III], gene_table, variable)

DataFrame(gene_table, columns=["variable", "sigA=off (I)", "glucose=off (II)", "glucose=on (III)"])

,variable,sigA=off (I),glucose=off (II),glucose=on (III)
0,ack,{'on': 100},{'off': 100},{'both': 100}
1,adc,{'off': 100},{'off': 100},"{'off': 50, 'both': 50}"
2,bdhAB,{'off': 100},{'off': 100},"{'off': 50, 'both': 50}"
3,buk1,{'on': 100},{'off': 100},{'both': 100}
4,crt,{'on': 100},{'off': 100},{'both': 100}
5,hbd,{'on': 100},{'off': 100},{'both': 100}
6,pta,{'on': 100},{'off': 100},{'both': 100}


Here we see again that ruling our sporulation in mode **I** leaves us with a much more "stable" profile in this mode. But for the remaining modes, the results remain unchanged compared to the original model. At this point, we could conclude that `f_2` and `f_3` simply play no significant role in the behaviour of our model, and we need more evidence to conclusively select either function.

However, we can consider one more criterium in selecting the update functions. In the absence of other experimental data, assuming our model is in mode **III**, we would expect the model to describe a complex behaviour when almost the whole pathway is active. Consequently, this means that the attractors should be fairly large in terms of state (vertex) count. To asses this, we can test how the admissible attractor states change when we fix different output values of `f_2` and `f_3`. 

To assess this, we can look at the number of attractor states that are valid when we set a specific row of the function table of `f_2` and `f_3`. For each bit of both function tables, we compute the "impact" of that particular row on the number of attractor states. Here, `100` means that the sets of attractor states appearing when `row=true` and `row=false` are fully disjoint, whereas impact `0` means that fixing this `row` will not change the attractor states in any way. Furthermore, we will be looking at `true`/`false` distribution *skew*: here, the two numbers together give `100%` and represent a preference towards a particular value. That is, selecting the output value with a higher skew will result in more attractor states, proprotionally to the previously mentioned "impact".

In [26]:
def check_attractor_size(graph, function, row, attractor):
    row_true = graph.mk_function_row_colors(function, row, True)
    row_false = graph.mk_function_row_colors(function, row, False)
    
    attractor_vertices_true = attractor.intersect_colors(row_true).vertices()
    attractor_vertices_false = attractor.intersect_colors(row_false).vertices()
    
    only_true = attractor_vertices_true.minus(attractor_vertices_false)
    only_false = attractor_vertices_false.minus(attractor_vertices_true)
    
    all_size = only_true.cardinality() + only_false.cardinality()
    if not int(all_size) == 0:
        true_percent = 100.0 * only_true.cardinality() / all_size
        false_percent = 100.0 * only_false.cardinality() / all_size
        impact = 100.0 * all_size / attractor.vertices().cardinality()

        print([int(x) for x in row], { "true": true_percent, "false": false_percent, "impact": impact })
    else:
        print([int(x) for x in row], " - no influence")

def table_rows(arity):
    """Build a list of input vectors representing the rows of a function table."""
    if arity == 0:
        return [[]]
    else:
        rows = table_rows(arity - 1)
        return [(x + [True]) for x in rows] + [(x + [False]) for x in rows]

print("Impact of f_2: ")
for row in table_rows(2):
    check_attractor_size(graph, "f_2", row, mode_III)

print("=====")
print("Impact of f_3: ")
for row in table_rows(3):
    check_attractor_size(graph, "f_3", row, mode_III)

Impact of f_2: 
[1, 1] {'true': 0.0, 'false': 100.0, 'impact': 0.007849700778291481}
[0, 1] {'true': 0.0, 'false': 100.0, 'impact': 0.007849700778291481}
[1, 0] {'true': 99.99816087068469, 'false': 0.0018391293153125906, 'impact': 0.007849299545830602}
[0, 0] {'true': 0.0018391293153125906, 'false': 99.99816087068469, 'impact': 0.007849299545830602}
=====
Impact of f_3: 
[1, 1, 1] {'true': 100.0, 'false': 0.0, 'impact': 0.07083481371332444}
[0, 1, 1]  - no influence
[1, 0, 1] {'true': 100.0, 'false': 0.0, 'impact': 100.0}
[0, 0, 1]  - no influence
[1, 1, 0]  - no influence
[0, 1, 0] {'true': 0.0, 'false': 100.0, 'impact': 100.0}
[1, 0, 0] {'true': 100.0, 'false': 0.0, 'impact': 99.89876809810677}
[0, 0, 0]  - no influence


First, let us look at the function `f_2`. Compared to `f_3`, it clearly has smaller impact on the number of attractor states (only `0.008%` of attractor states depend on any row of this function). Now, see that `f_2[0,1]` and `f_2[1,1]` are "perfectly skewed" towards the value `false` (i.e. there is no benefit in picking `true` in terms of attractor size). This effectively means that to maximise the number of attractor states, we should fix `f_2(x_0, x_1) = !x_1 & f_2'(x_0)`. Finally, to determine `f_2'`, we look at `f_2[0,0]` and `f_2[1,0]`. Given our current setting, the choice of these two logical parameters effectively dictates whether `cell_membrane` regulates `glucose___PTS` positively or negatively, with the skew suggesting that positive regulation leads to more attractor states. Consequently, we end up with `f_2(x_0, x_1) = x_0 & !x_1`, giving us the update function for `glucose___PTS` as `glucose & PTS & cell_membrane & !saturation`.

Now, consider function `f_3`. Here, we have two values (`f_3[0,1,0] = false` and `f_3[1,0,1] = true`) with `impact: 100` and a "perfect skew": these function rows simply need to be fixed if we want to obtain a function that is consistent with the regulatory graph at all. 

Notice that all the remaining function rows also have perfect skew, only the impact is reduced. This means that selecting the suggested output value will not eliminate any attractor states. However, note that even if we fix all the values as suggested, we still have several function rows with ambiguous values that have no influence on the number of attractor states. 

To resolve this ambiguity, we look at the original (inconsistent) update function: the closest modification which also satisfies all the requirements we discussed here gives us `f_3(x_0, x_1, x_2) = x_0 & (!x_1 | x_2)`, which results in the update function `spo0A_p & sigH & (spoIIE | !spoIIAA)`. But note that there are several other possible functions which produce exactly the same attractor states.

In [27]:
# Reload the model again
model = BooleanNetwork.from_file('butanol-pathway.sbml')

# Fix update functions using the repaired values
f_spo0A_p = "spo0A & phosphorylation & sigA & !sporulation"
f_glucose_PTS = "glucose & PTS & cell_membrane & !sporulation"
f_spoIIAB = "spo0A_p & sigH & (spoIIE | !spoIIAA)"

model.set_update_function("spo0A_p", f_spo0A_p)
model.set_update_function("glucose___PTS", f_glucose_PTS)
model.set_update_function("spoIIAB", f_spoIIAB)

Path('butanol-pathway-repaired.sbml').write_text(model.to_sbml());

### Discussion

In this case study, we expanded on the results presented in \[1]. Specifically, we perform a full attractor analysis of the proposed signalling pathway. We show that out of the `13` available logical parameters, the long-term behaviour of the model is primarily sensitive to two: `glucose` and `sigA`. We then cross-reference this attractor analysis with the simulation results and in-vivo measurements, including the stability of pathway products (`butanol`, `acetone`, `ethanol`, and other substances measured in \[1]). We show that the attractors of the signalling pathway are indeed consistent with the known measurements and simulation. 

Additionally, we uncovered a small percentage of parametrisations which enable intermittent `ethanol` production. Production of `ethanol` was observed in small quantities during in-vivo measurements, but was not reproduced in network simulations. Now we have a plausible explanation for this behaviour. Similarly, for `lactic_acid`, which was the second least abundant substance during in-vivo measurements (after `ethanol`), we see a significant portion of parametrisations (`52%`) with `lactic_acid` production disabled. This suggests the network is correctly reflecting the real-world dynamics of these variables. 

However, we found that in the absence of `sigA`, the network admits various biologically unrealistic outcomes. In particular, sporulation can be active while the pythway is producing products. We have also identified several logical inconsistencies between network's update functions and its regulatory graph. We thus attempted to repair the network by parametrising its problematic update functions. 

We identified a single instantiation of the `spo0A+p` update function which both reduces unrealistic behaviour when `sigA` is inactive and preserves the agreement with in-vivo measurements. To identify the remaining update functions which do not seem to influence the network products, we analysed their impact on the total number of attractor states. Here, we discovered that the update function of `spoIIAB` has significant impact on the number of attractor states (`99.9%` of such states can be eliminated by the choice of this function). Subsequently, we selected instantiations for `spoIIAB` and `glucose___PTS` functions which maximise attractor states. We provide this repaired network in `butanol-pathway-repaired.sbml`. 

### References

\[1] Jana Musilová. “Signaling Pathway for Butanol Produc- tion in Solventogenic Clostridium Bacteria.” MA thesis. Brno University of Technology, 2019.

\[2] Patakova, Petra, et al. "Acidogenesis, solventogenesis, metabolic stress response and life cycle changes in Clostridium beijerinckii NRRL B-598 at the transcriptomic level." Scientific reports 9.1 (2019): 1-21.

\[3] Sedlar, Karel, et al. "Transcription profiling of butanol producer Clostridium beijerinckii NRRL B-598 using RNA-Seq." BMC genomics 19.1 (2018): 1-13.

\[4] Al-Hinai, Mohab A., Shawn W. Jones, and Eleftherios T. Papoutsakis. "The Clostridium sporulation programs: diversity and preservation of endospore differentiation." Microbiology and Molecular Biology Reviews 79.1 (2015): 19-37.